In [4]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

parameters["form_compiler"]["optimize"]     = True
parameters["form_compiler"]["cpp_optimize"] = True

# Problem data

Most of this is ignored for now.

In [5]:
class InitialInPlaneDisplacements(Expression):     
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = -x[1]/10.0
    def value_shape(self):
        return (2,)

class InitialOutOfPlaneDisplacements(Expression):
    def eval(self, values, x):
        #values[0] = 0.5*x[0]**2*(1-x[0])**2*sin(4*DOLFIN_PI*x[1])
        values[0] = x[0]*(1-x[0])*(1-2*x[0])*sin(4*DOLFIN_PI*x[1])
        values[1] = 2*DOLFIN_PI*x[0]**2*(1-x[0])**2*cos(4*DOLFIN_PI*x[1])
    def value_shape(self):
        return (2,)

In [6]:
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0]) < DOLFIN_EPS and on_boundary

# Discrete spaces

In-plane displacements (IPD) are discretized with $(P_2)^2$. Out-of-plane displacements with a mixed function space $P_2 \ast (P_1)^2$.

In [ ]:
msh = RectangleMesh(Point(0.0, -0.5), Point(1.0, 0.5), 20, 20)

U = VectorFunctionSpace(msh, "Lagrange", 1, dim=2)        # in plane displacements (IPD)
V = VectorFunctionSpace(msh, "Lagrange", 1, dim=2)        # Gradients of out of plane displacements (OPD)

u  = Function(U)
v  = Function(V)
u_ = Function(U)    # solution from previous step
v_ = Function(V)    # solution from previous step

u_init = InitialInPlaneDisplacements(degree=1)
u_.interpolate(u_init)

v_init = InitialOutOfPlaneDisplacements(degree=1)
v_.interpolate(v_init)

bcU = DirichletBC(U, Expression(("0.0", "-x[1]/10"), element=U.ufl_element()), DirichletBoundary())
bcV = DirichletBC(V, Constant((0.0, 0.0)), DirichletBoundary())

In [ ]:
pl.figure(figsize=(7,7))
_ = plot(v_, title="Initial OPD")

# Coordinate descent

In [ ]:
def eps(u):
    return (grad(u) + grad(u).T)/2.0

e_stop = msh.hmin()*1e-3

#set_log_level(PROGRESS)
file = File("displacements.pvd")

mu = Constant(1e4)

Id = Identity(2)

phi = TestFunction(U)
psi = TestFunction(V)
dtu = TrialFunction(U)
dtv = TrialFunction(V)

Lu = inner(dtu, phi)*dx
Lv = inner(dtv, psi)*dx

In [ ]:
zero_energy = assemble((1./24)*inner(Id, Id)*dx(msh))
def energy(u, v, mu=mu):
    J = (1./2)*inner(eps(u)+outer(v, v)/2, eps(u)+outer(v, v)/2)*dx \
        + (1./24)*inner(grad(v) - Id, grad(v) - Id)*dx \
        + mu*inner(v, v)*dx
    return assemble(J)

In [ ]:
du = Function(U)
dv = Function(V)
step = 0
tau = 1.0
omega = 0.25

cur_energy = energy(u, v)

while step < 10:
    print("Step %d, energy = %e" % (step, cur_energy))
    #This of course won't work
    dJu = inner(eps(u_)+outer(v_, v_)/2, eps(phi))*dx
    dJv = inner(eps(u_)+outer(v_, v_)/2, outer(v_, psi))*dx \
          + (1./12)*inner(grad(v_)-Id, grad(psi))*dx \
          + 2*mu*inner(v_, psi)*dx

    solve(Lu == -dJu, du, bcU)
    solve(Lv == -dJv, dv, bcV)
    
    ndu = norm(du.vector())
    ndv = norm(dv.vector())
    
    print("|du| = %.3f" % ndu)
    print("|dv| = %.3f" % ndv)

    ## line search
    new_energy = 0
    alpha = tau
    while True:
        u = project(u_ + alpha*du, U)
        v = project(v_ + alpha*dv, V)
        new_energy = energy(u, v)
        print("New energy: %e, alpha = %.2e" % (new_energy, alpha))
        if new_energy <= cur_energy - omega*alpha*(ndu**2+ndv**2):
            print("Line search ended with alpha = %e" % alpha)
            cur_energy = new_energy
            break
        if alpha < (1./2)**30:
            raise Exception("Line search failed after 30 steps")
        alpha *= 0.5
    
    file << (u, tau)
    file << (v, tau)
    
    u_.vector()[:] = u.vector()
    v_.vector()[:] = v.vector()
    
    step += 1

# Gradient descent

In [10]:
class InitialDisplacements(Expression):     
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = -x[1]/10.0
        values[2] = x[0]*(1-x[0])*(1-2*x[0])*sin(4*DOLFIN_PI*x[1])
        values[3] = 2*DOLFIN_PI*x[0]**2*(1-x[0])**2*cos(4*DOLFIN_PI*x[1])
    def value_shape(self):
        return (4,)

msh = RectangleMesh(Point(0.0, -0.5), Point(1.0, 0.5), 20, 20)

UE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)        # in plane displacements (IPD)
VE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)        # Gradients of out of plane displacements (OPD)
W = FunctionSpace(msh, UE*VE)

bcU = DirichletBC(W.sub(0), Expression(("0.0", "-x[1]/10"), element=UE), DirichletBoundary())
bcV = DirichletBC(W.sub(1), Constant((0.0, 0.0)), DirichletBoundary())

w = Function(W)
w_ = Function(W)
u, v  = w.split()
u_, v_ = w_.split()

w_init = InitialDisplacements(degree=1)
w.interpolate(w_init)
w_.interpolate(w_init)


def eps(u):
    return (grad(u) + grad(u).T)/2.0

e_stop = msh.hmin()*1e-4

file = File("steepest-descent.pvd")

mu = Constant(1e4)

Id = Identity(2)

z = TestFunction(W)
dtz = TrialFunction(W)
L = inner(dtz, z)*dx

zero_energy = assemble((1./24)*inner(Id, Id)*dx(msh))
def energy(u, v, mu=mu):
    J = (1./2)*inner(eps(u)+outer(v, v)/2, eps(u)+outer(v, v)/2)*dx \
        + (1./24)*inner(grad(v) - Id, grad(v) - Id)*dx \
        + mu*inner(v, v)*dx
    return assemble(J)

phi,psi = TestFunctions(W)

dw = Function(W)
du, dv = dw.split()
step = 0
tau = 2**-5
omega = 0.25

cur_energy = energy(u, v)

alpha = ndu = ndv = 1.0
history = {'J':[], 'alpha':[]}

while alpha*(ndu**2+ndv**2) > e_stop and step < 1000:
    print("Step %d, energy = %.3e, curl = %.3e" % (step, cur_energy, assemble(curl(w.sub(1))*dx)))
    dJ = inner(eps(u_)+outer(v_, v_)/2, eps(phi))*dx \
        + inner(eps(u_)+outer(v_, v_)/2, outer(v_, psi))*dx \
        + (1./12)*inner(grad(v_)-Id, grad(psi))*dx \
        + 2*mu*inner(v_, psi)*dx

    print("\tSolving...", end='')
    solve(L == -dJ, dw, [bcU, bcV])
    
    # dw is not assigned to a new object so it's ok to reuse du, dv without splitting
    ndu = norm(du)
    ndv = norm(dv)
    
    print(" done with |du| = %.3f, |dv| = %.3f" % (ndu, ndv), end='')

    # line search
    new_energy = 0
    alpha = tau
    while True:
        w = project(w_ + alpha*dw, W)
        u, v = split(w)
        new_energy = energy(u, v)
        #print("New energy: %e, alpha = %.2e" % (new_energy, alpha))
        if new_energy <= cur_energy - omega*alpha*(ndu**2+ndv**2):
            print(", alpha = %.3e" % alpha)
            history['J'].append(cur_energy)
            history['alpha'].append(alpha)
            cur_energy = new_energy
            break
        if alpha < (1./2)**30:
            raise Exception("Line search failed after 30 steps")
        alpha *= 0.5
    w.rename("disp", "displacement")
    file << (w.split()[0], float(step))
    file << (w.split()[1], float(step))
    
    w_.vector()[:] = w.vector()
    u_, v_ = w_.split()

    step += 1

Step 0, energy = 3.142e+02, curl = -4.283e-18
	Solving... done with |du| = 0.951, |dv| = 3546.015, alpha = 6.104e-05
Step 1, energy = 1.552e+01, curl = 6.017e-06
	Solving... done with |du| = 1.039, |dv| = 785.836, alpha = 6.104e-05
Step 2, energy = 8.458e-01, curl = -2.952e-06
	Solving... done with |du| = 0.925, |dv| = 174.149, alpha = 6.104e-05
Step 3, energy = 1.253e-01, curl = 1.087e-06
	Solving... done with |du| = 0.830, |dv| = 38.594, alpha = 6.104e-05
Step 4, energy = 8.992e-02, curl = -3.556e-07
	Solving... done with |du| = 0.760, |dv| = 8.553, alpha = 6.104e-05
Step 5, energy = 8.815e-02, curl = 1.092e-07
	Solving... done with |du| = 0.708, |dv| = 1.895, alpha = 6.104e-05
Step 6, energy = 8.803e-02, curl = -3.221e-08
	Solving... done with |du| = 0.668, |dv| = 0.420, alpha = 1.221e-04
Step 7, energy = 8.798e-02, curl = 5.075e-08
	Solving... done with |du| = 0.608, |dv| = 0.606, alpha = 1.221e-04
Step 8, energy = 8.795e-02, curl = -7.967e-08
	Solving... done with |du| = 0.569, |d

	Solving... done with |du| = 0.261, |dv| = 0.150, alpha = 2.441e-04
Step 72, energy = 8.674e-02, curl = -1.576e-05
	Solving... done with |du| = 0.259, |dv| = 0.616, alpha = 6.104e-05
Step 73, energy = 8.673e-02, curl = 4.362e-06
	Solving... done with |du| = 0.259, |dv| = 0.171, alpha = 1.221e-04
Step 74, energy = 8.672e-02, curl = -6.778e-06
	Solving... done with |du| = 0.258, |dv| = 0.265, alpha = 1.221e-04
Step 75, energy = 8.671e-02, curl = 1.053e-05
	Solving... done with |du| = 0.257, |dv| = 0.412, alpha = 6.104e-05
Step 76, energy = 8.671e-02, curl = -2.917e-06
	Solving... done with |du| = 0.257, |dv| = 0.114, alpha = 2.441e-04
Step 77, energy = 8.669e-02, curl = 1.198e-05
	Solving... done with |du| = 0.255, |dv| = 0.470, alpha = 6.104e-05
Step 78, energy = 8.669e-02, curl = -3.320e-06
	Solving... done with |du| = 0.255, |dv| = 0.130, alpha = 2.441e-04
Step 79, energy = 8.668e-02, curl = 1.364e-05
	Solving... done with |du| = 0.253, |dv| = 0.535, alpha = 6.104e-05
Step 80, energy 

	Solving... done with |du| = 0.217, |dv| = 0.539, alpha = 6.104e-05
Step 143, energy = 8.621e-02, curl = 3.746e-06
	Solving... done with |du| = 0.217, |dv| = 0.150, alpha = 1.221e-04
Step 144, energy = 8.620e-02, curl = -5.827e-06
	Solving... done with |du| = 0.216, |dv| = 0.233, alpha = 1.221e-04
Step 145, energy = 8.620e-02, curl = 9.064e-06
	Solving... done with |du| = 0.216, |dv| = 0.362, alpha = 6.104e-05
Step 146, energy = 8.619e-02, curl = -2.518e-06
	Solving... done with |du| = 0.216, |dv| = 0.101, alpha = 2.441e-04
Step 147, energy = 8.619e-02, curl = 1.035e-05
	Solving... done with |du| = 0.215, |dv| = 0.414, alpha = 6.104e-05
Step 148, energy = 8.618e-02, curl = -2.876e-06
	Solving... done with |du| = 0.215, |dv| = 0.115, alpha = 2.441e-04
Step 149, energy = 8.617e-02, curl = 1.182e-05
	Solving... done with |du| = 0.214, |dv| = 0.473, alpha = 6.104e-05
Step 150, energy = 8.617e-02, curl = -3.285e-06
	Solving... done with |du| = 0.214, |dv| = 0.131, alpha = 2.441e-04
Step 151

In [ ]:
pl.plot(np.log(history['J'][5:]))
_ = pl.title('$log\ J$')

In [ ]:
w.interpolate(w_init)

P = FunctionSpace(msh, "Lagrange", 1)
bcP = DirichletBC(P, Constant(0.0), DirichletBoundary())

p = TrialFunction(P)
q = TestFunction(P)
z = TestFunction(P)

a = inner(p.dx(0), q)*dx + inner(p.dx(1), z)*dx
L = inner(w.split()[1].sub(0), q)*dx + inner(w.split()[1].sub(1), z)*dx
opd = Function(P)
solve(a == L, opd, bcP)

In [ ]:
plot(opd)

In [ ]:
orig = Function(W)
orig.interpolate(wdw_init)

diff = project(orig - dwk)
print(norm(diff.vector(), 'l2'))
print(norm(diff.vector(), 'linf'))

In [ ]:
pl.figure(figsize=(22,8))
pl.subplot(1,3,1)
_ = plot(orig, title="Init: %e - %e" % (orig.vector().min(), orig.vector().max()))
pl.subplot(1,3,2)
_ = plot(dwk, title="Sol: %e - %e" % (dwk.vector().min(), dwk.vector().max()))
pl.subplot(1,3,3)
_ = plot(diff, title="Diff: %e - %e" % (diff.vector().min(), diff.vector().max()))

# curl constraint

In [ ]:
class InitialDisplacements(Expression):     
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = -x[1]/10.0
        values[2] = x[0]*(1-x[0])*(1-2*x[0])*sin(4*DOLFIN_PI*x[1])
        values[3] = 2*DOLFIN_PI*x[0]**2*(1-x[0])**2*cos(4*DOLFIN_PI*x[1])
    def value_shape(self):
        return (4,)

msh = RectangleMesh(Point(0.0, -0.5), Point(1.0, 0.5), 20, 20)

UE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)        # in plane displacements (IPD)
VE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)        # Gradients of out of plane displacements (OPD)
W = FunctionSpace(msh, UE*VE)

bcU = DirichletBC(W.sub(0), Expression(("0.0", "-x[1]/10"), element=UE), DirichletBoundary())
bcV = DirichletBC(W.sub(1), Constant((0.0, 0.0)), DirichletBoundary())

w = Function(W)
w_ = Function(W)
u, v  = w.split()
u_, v_ = w_.split()

w_init = InitialDisplacements(degree=1)
w.interpolate(w_init)
w_.interpolate(w_init)


def eps(u):
    return (grad(u) + grad(u).T)/2.0

e_stop = msh.hmin()*1e-4

file = File("steepest-descent-curl.pvd")

mu = Constant(1e4)
Id = Identity(2)


zero_energy = assemble((1./24)*inner(Id, Id)*dx(msh))
def energy(u, v, mu=mu):
    J = (1./2)*inner(eps(u)+outer(v, v)/2, eps(u)+outer(v, v)/2)*dx \
        + (1./24)*inner(grad(v) - Id, grad(v) - Id)*dx \
        + mu*inner(curl(v), curl(v))*dx
    return assemble(J)

z = TestFunction(W)
dtz = TrialFunction(W)
L = inner(dtz, z)*dx

phi,psi = TestFunctions(W)

dw = Function(W)
du, dv = dw.split()
step = 0
tau = 2**-5
omega = 0.25

cur_energy = energy(u, v)

alpha = ndu = ndv = 1.0
history = {'J':[], 'alpha':[]}

while alpha*(ndu**2+ndv**2) > e_stop and step < 100:
    print("Step %d, energy = %.3e, curl = %.3e" % (step, cur_energy, assemble(curl(v_)*dx)))
    dJ = inner(eps(u_)+outer(v_, v_)/2, eps(phi))*dx \
        + inner(eps(u_)+outer(v_, v_)/2, outer(v_, psi))*dx \
        + (1./12)*inner(grad(v_)-Id, grad(psi))*dx \
        + 2*mu*inner(curl(v_), curl(psi))*dx

    print("\tSolving...", end='')
    solve(L == -dJ, dw, [bcU, bcV])
    
    # dw is not assigned to a new object so it's ok to reuse du, dv without splitting
    ndu = norm(du)
    ndv = norm(dv)
    
    print(" done with |du| = %.3f, |dv| = %.3f" % (ndu, ndv), end='')

    # line search
    new_energy = 0
    alpha = tau
    while True:
        w = project(w_ + alpha*dw, W)
        u, v = split(w)
        new_energy = energy(u, v)
        #print("New energy: %e, alpha = %.2e" % (new_energy, alpha))
        if new_energy <= cur_energy - omega*alpha*(ndu**2+ndv**2):
            print(", alpha = %e" % alpha)
            history['J'].append(cur_energy)
            history['alpha'].append(alpha)
            cur_energy = new_energy
            break
        if alpha < (1./2)**30:
            raise Exception("Line search failed after 30 steps")
        alpha *= 0.5
    w.rename("disp", "displacement")
    file << (w.split()[0], float(step))
    file << (w.split()[1], float(step))
    
    w_.vector()[:] = w.vector()
    u_, v_ = w_.split()

    step += 1

# Mixed out-of-plane displacements

In [8]:
class InitialDisplacements(Expression):     
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = -x[1]/10.0
        values[2] = 0.5*x[0]**2*(1-x[0])**2*sin(4*DOLFIN_PI*x[1])
        values[3] = x[0]*(1-x[0])*(1-2*x[0])*sin(4*DOLFIN_PI*x[1])
        values[4] = 2*DOLFIN_PI*x[0]**2*(1-x[0])**2*cos(4*DOLFIN_PI*x[1])
    def value_shape(self):
        return (5,)

msh = RectangleMesh(Point(0.0, -0.5), Point(1.0, 0.5), 20, 20)

UE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)        # in plane displacements (IPD)
VE = FiniteElement("Lagrange", msh.ufl_cell(), 1)
VVE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)        # Gradients of out of plane displacements (OPD)
ME = MixedElement([UE, VE, VVE])
W = FunctionSpace(msh, ME)

bc = DirichletBC(W, Expression(("0.0", "-x[1]/10", "0.0", "0.0", "0.0"), element=ME),
                 DirichletBoundary())
w = Function(W)
w_ = Function(W)
u, d, v  = w.split()
u_, d_, v_ = w_.split()

w_init = InitialDisplacements(degree=1)
w.interpolate(w_init)
w_.interpolate(w_init)

def eps(u):
    return (grad(u) + grad(u).T)/2.0

e_stop = msh.hmin()*1e-4

file = File("steepest-descent-grad.pvd")

mu = Constant(1e4)
Id = Identity(2)

zero_energy = assemble((1./24)*inner(Id, Id)*dx(msh))
def energy(u, d, v, mu=mu):
    J = (1./2)*inner(eps(u)+outer(v, v)/2, eps(u)+outer(v, v)/2)*dx \
        + (1./24)*inner(grad(v) - Id, grad(v) - Id)*dx \
        + (1./2)*mu*inner(v, v)*dx \
        + (1./2)*mu*inner(v - grad(d), v - grad(d))*dx
        
    return assemble(J)

phi, eta, psi = TestFunctions(W)

dtz = TrialFunction(W)
z = TestFunction(W)
L = inner(dtz, z)*dx

dw = Function(W)
du, dd, dv = dw.split()
step = 0
max_line_steps = 30
tau = 2**-50
omega = 0.25  # fudge factor

cur_energy = energy(u, d, v)

alpha = ndu = ndd = ndv = 1.0
history = {'J':[], 'alpha':[]}

while alpha*(ndu**2+ndd**2+ndv**2) > e_stop and step < 100:
    print("Step %d, energy = %.3e, curl = %.3e" % (step, cur_energy, assemble(curl(w.sub(2))*dx)))
    dJ = inner(eps(u_) + outer(v_, v_)/2, eps(phi))*dx \
        + inner(eps(u_) + outer(v_, v_)/2, outer(v_, psi))*dx \
        + (1./12)*inner(grad(v_)-Id, grad(psi))*dx \
        + mu*inner(v_, psi)*dx \
        + mu*inner(v_ - grad(d_), phi)*dx + mu*inner(v_ - grad(d_), grad(eta))*dx

    print("\tSolving...", end='')
    solve(L == -dJ, dw, bc)
    
    # dw is not assigned to a new object so it's ok to reuse du, dd, dv without splitting
    ndu = norm(du)
    ndd = norm(dd)
    ndv = norm(dv)
    
    print(" done with |du| = %.3f, |dd| = %.3f, |dv| = %.3f" % (ndu, ndd, ndv), end='')

    # line search
    new_energy = 0
    alpha = tau
    while True:
        w = project(w_ + alpha*dw, W)
        u, d, v = split(w)
        new_energy = energy(u, d, v)
        #print("New energy: %e, alpha = %.2e" % (new_energy, alpha))
        if new_energy <= cur_energy - omega*alpha*(ndu**2+ndd**2+ndv**2):
            print(", alpha = %e" % alpha)
            history['J'].append(cur_energy)
            history['alpha'].append(alpha)
            cur_energy = new_energy
            break
        if alpha < (1./2)**max_line_steps:
            raise Exception("Line search failed after %d steps" % max_line_steps)
        alpha *= 0.5
    w.rename("disp", "displacement")
    file << (w.split()[0], float(step))
    file << (w.split()[1], float(step))
    
    w_.vector()[:] = w.vector()
    u_, d_, v_ = w_.split()

    step += 1

Step 0, energy = 1.640e+02, curl = -4.283e-18
	Solving... done with |du| = 25.824, |dd| = 1799.638, |dv| = 1774.325

Exception: Line search failed after 30 steps